# Phonetic rules, take 2

> "Second attempt at an earlier notebook"

- toc: false
- branch: master
- hidden: true
- categories: [swedish, nst, lexicon, phonetic, rules]

Earlier notebook is [here]({% post_url 2023-10-09-phonetic-rule-take-1 %})

In [2]:
_ALPHABET = {
    "A": ["a"],
    "B": ["be", "bé"],
    "C": ["ce", "se", "sé", "cé", "si", "ci"],
    "D": ["de", "dé", "di"],
    "E": ["e", "é"],
    "F": ["eff", "ef"],
    "G": ["ge", "gé", "gi"],
    "H": ["hå", "ho"],
    "I": ["i"],
    "J": ["ji", "gi"],
    "K": ["kå", "ko"],
    "L": ["ell", "el"],
    "M": ["emm", "em"],
    "N": ["enn", "en"],
    "O": ["o"],
    "P": ["pe", "pé", "pi"],
    "Q": ["qu"],
    "R": ["err", "er", "är", "ärr"],
    "S": ["ess", "es"],
    "T": ["te", "té", "ti"],
    "U": ["u"],
    "V": ["ve", "vé", "vi"],
    "W": ["dubbelve"],
    "X": ["ex", "ecz", "ecs", "eks"],
    "Y": ["y"],
    "Z": ["zäta", "säta", "seta", "zeta"],
    "Å": ["å"],
    "Ä": ["ä"],
    "Ö": ["ö"]
}

DIGITS = {
    "1": ["ett"],
    "2": ["två"],
    "3": ["tre"],
    "4": ["fyra"],
    "5": ["fem"],
    "6": ["sex"],
    "7": ["sju"],
    "8": ["åtta"],
    "9": ["nio", "ni"]
}

In [3]:
ALPHABET = {k: sorted(v, key=len, reverse=True) for k,v in _ALPHABET.items()}

```
2442206120015761721 1 487.78 0.32 essefu 1.0 <eps> ins
2442206120015761721 1 488.26 0.16 fem 1.0 SfU5 sub
```

```
2442206120015761721 1 490.6 0.519 tjugohundrafjorton 1.0 <eps> ins
2442206120015761721 1 491.2 0.379 femton 1.0 <eps> ins
2442206120015761721 1 491.74 0.779 etthundratjugofyra 1.0 2014/15:124 sub
```

In [4]:
ALNUM = {**ALPHABET, **DIGITS}
ALNUM_REGEX = {k: f"({'|'.join(v)})" for k,v in ALNUM.items()}

In [5]:
from difflib import SequenceMatcher

ACCEPT = [
    ("e", "ə")
]

a = "kamaren"
b = "kamarən"

ok = False

s = SequenceMatcher(None, a, b)
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag == "replace":
        if (a[i1:i2], b[j1:j2]) in ACCEPT:
            ok = True
        else:
            ok = False
    elif tag == "equal":
        ok = True

In [6]:
from phonemizer import phonemize

In [7]:
phonemize("slut", language='sv')

'slʉt '

In [8]:
from sync_asr.utils.nst_lexicon import get_nst_lexicon

In [9]:
%cd /Users/joregan/Playing/sync_asr
%pip install -e .

/Users/joregan/Playing/sync_asr
Obtaining file:///Users/joregan/Playing/sync_asr
  Preparing metadata (setup.py) ... done
  Attempting uninstall: sync-asr
    Found existing installation: sync-asr 0.0.1
    Uninstalling sync-asr-0.0.1:
      Successfully uninstalled sync-asr-0.0.1
  Running setup.py develop for sync-asr
Note: you may need to restart the kernel to use updated packages.


In [10]:
lexicon = get_nst_lexicon()

In [11]:
dictionary = {}
for entry in lexicon:
    if 'garbage' in entry and entry['garbage'] == 'GARB':
        continue
    else:
        word = entry['orthography']
        word = word.replace("_", " ")
        if not word in dictionary:
            dictionary[word] = set()
        for translit in entry['transliterations']:
            dictionary[word].add(translit['ipa'].replace(".", "").replace("¤", "").replace("_", " ").replace("\u0361", ""))

In [12]:
CONSONANTS = "bdfhjklmnprstvŋɕɖɡɧɭɳʂʈ"
VOWELS = "aeiouyøɪɑɔɛɵʉʊʏ"
ACCENTS = "²ˌˈ"

In [13]:
print(dictionary["inte"])
print(phonemize("inte", language='sv'))

{'²ɪnte'}
ɪntɛ 


In [14]:
characters = set()
for entry in dictionary:
    for pron in dictionary[entry]:
        for char in pron:
            characters.add(char)

In [15]:
cons = set()
VS = VOWELS + ACCENTS + "_"
for entry in dictionary:
    for pron in dictionary[entry]:
        for char in pron:
            if char not in VS:
                cons.add(char)

In [16]:
def shift_accent(ipa):
    accent = ""
    output = ""
    for character in ipa:
        if character in ACCENTS:
            accent = character
        elif character in CONSONANTS:
            output += character
        elif character in VOWELS:
            if accent != "":
                output += accent
                accent = ""
            output += character
        else:
            output += character
    return output

In [17]:
shift_accent("²kratʊʂ")

'kr²atʊʂ'

In [18]:
import re
import itertools

class Rule():
    def __init__(self, match, replacement, rulename, example, on_accented = False, before_accented = False):
        self.match = match
        self.replacement = replacement
        self.rulename = rulename
        self.example = example
        self.on_accented = on_accented
        self.before_accented = before_accented

    def apply(self, word):
        if self.on_accented:
            pattern = fr"((?:[^²ˌˈ]){self.match}|^{self.match})"
        elif self.before_accented:
            pattern = fr"({self.match}(?:[^²ˌˈ]))"
        else:
            word = re.sub("[²ˌˈ]", "", word)
            pattern = self.match
        matches = [(m.start(), m.end()) for m in re.finditer(pattern, word)]
        if self.on_accented:
            tmp = []
            for m in matches:
                if m[1] - m[0] > len(self.match):
                    tmp.append((m[1] - len(self.match), m[1]))
                else:
                    tmp.append(m)
            matches = tmp

        pieces = []
        prev_end = 0
        for piece in matches:
            if piece[0] > 0:
                pieces.append([word[prev_end:piece[0]]])
            pieces.append([word[piece[0]:piece[1]], self.replacement])
            prev_end = piece[1]
        pieces.append([word[prev_end:]])

        output = []
        for part in itertools.product(*pieces):
            output.append("".join(part))
        return output

In [19]:
rule = Rule("nh", "n", "n → ∅ / _ h", "Stenholm")

In [20]:
assert rule.apply("nhanhanha") == ['nhanhanha',
 'nhanhana',
 'nhananha',
 'nhanana',
 'nanhanha',
 'nanhana',
 'nananha',
 'nanana']

In [21]:
# The basic idea with these is that they just apply anyway, because the input
# is single words, but at some later processing stage we can validate
class AssimilationRule(Rule):
    def __init__(self, match, replacement, rulename, example, on_accented = False, before_accented = False, pre_context = "", post_context = ""):
        super.__init__(match, replacement, rulename, example, on_accented, before_accented)
        self.pre_context = pre_context
        self.post_context = post_context

In [22]:
GENERAL_STRESSED = [
    Rule("e", "ə", "e → ə / [-stressed]", "", True),
    Rule("ɛ", "ə", "e → ə / [-stressed]", "", True)
]

AssimilationRule("r$", "", "h → ∅ / r # _", "har han", False, False, "", "h")
AssimilationRule("n$", "", "h → ∅ / n # _", "han har", False, False, "", "h")
AssimilationRule("r$", "", "r → ∅ / _ # [+consonant]", "där bilen", False, False, "", f"[{CONSONANTS}]")

Rule("[œɶeə]r", "r", "e → ∅ / _ r [+stressed]", "bero", False, True)
	
Rule("ɪntə", "ntə", "ɪ → ∅ / [+vowel] # _ n t ə", "ska inte", False, False)
Rule("ɪnte", "nte", "ɪ → ∅ / [+vowel] # _ n t e", "ska inte", False, False)

Rule("nh", "h", "n → ∅ / _ h", "Stenholm")

TypeError: descriptor '__init__' requires a 'super' object but received a 'str'

: 